In [11]:
import sys
sys.path.append("../")
print(sys.path)

['/home/varun/env/envis/lib/python3.8/site-packages/ray/thirdparty_files', '/home/varun/env/envis/lib/python3.8/site-packages/ray/pickle5_files', '/home/varun/EnvisEdge/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/varun/env/envis/lib/python3.8/site-packages', '/home/varun/.local/lib/python3.8/site-packages', '../', '../']


In [12]:
import yaml
import json
from json import dumps, loads
from kafka import KafkaProducer, KafkaConsumer
from fedrec.data_models.job_submit_model import JobSubmitMessage
from fedrec.utilities import registry
import fedrec
import experiments
import fl_strategies
import datasets
with open("../configs/regression.yml", 'r') as cfg:
    config = yaml.load(cfg, Loader=yaml.FullLoader)

def init_kafka(config):
    producer_url = "{}:{}".format(
        config["producer_url"], config["producer_port"])
    return KafkaProducer(
        bootstrap_servers=[producer_url])

# serializer = registry.construct("serializer", "json")
# config = config["multiprocessing"]["communications"]

producer = init_kafka(config["multiprocessing"]["communication_interface"])
producer.send('job-request-trainer', value=str({"job_type": "test_run","job_args": [1,2]}).encode('utf-8'))


In [2]:
with open("../configs/dlrm_fl.yml", 'r') as cfg:
    config = yaml.load(cfg, Loader=yaml.FullLoader)

ag_config = {
        # Seed for RNG used in shuffling the training data.
    "data_seed" : 100,
    # Seed for RNG used in initializing the model.
    "init_seed" : 100,
    # Seed for RNG used in computing the model's training loss.
    # Only relevant with internal randomness in the model, e.g. with dropout.
    "model_seed" : 100
}


FileNotFoundError: [Errno 2] No such file or directory: '../configs/dlrm_fl.yml'

In [3]:
from fedrec.utilities.serialization_utils import Serializable

class A(Serializable):
    def __init__(self, a, b):
        super().__init__()
        self.a = a
        self.b = b

class B(A):
    def __init__(self, a, b, c):
        super().__init__(a, b)
        self.c = c

In [4]:
obj = B(1,2,3)


In [5]:
obj.__class__.__bases__

(__main__.A,)

In [13]:
class A:
    def __init__(self):
        print("A")

class B(A):
    def __init__(self):
        print("B")
        super().__init__()
        
    def power(self):
        print("power")
        
class C(B):
    def __init__(self):
        print("C")
        super().__init__()
        

In [14]:
object = C()

C
B
A


In [2]:
dic = {'__type__': 'JobSubmitMessage', '__data__': {'job_type': 'train', 'job_args': [], 'job_kwargs': {}, 'senderid': 0, 'receiverid': 0, 'workerstate': {'__type__': 'TrainerState', '__data__': {'id': 0, 'round_idx': 0, 'state_dict': {'model': '/home/ramesht/dump_tensor/worker_id_0/0_0_trainer.pt', 'step': 10}, 'storage': '/home/ramesht/dump_tensor/worker_id_0', 'model_preproc': {'__type__': 'RegressionPreprocessor', '__data__': {'proc_config': {'name': 'regression'}, 'client_id': 0, 'dataset_config': {'name': 'femnist', 'data_dir': '/home/ubuntu/dataset/femnist', 'splits': ['train', 'test', 'val']}}}, 'local_sample_number': None, 'local_training_steps': 10}}}}

In [5]:
if "__type__" in dic:
    type = dic["__type__"]

print(type)

JobSubmitMessage
